[![Binder](../img/badge-binder.svg)](https://mybinder.org/v2/gh/nhirschey/teaching/gh-pages?filepath=assignments/assignment-performance-evaluation.ipynb)&emsp;
[![Script](../img/badge-script.svg)](/Teaching//assignments/assignment-performance-evaluation.fsx)&emsp;
[![Notebook](../img/badge-notebook.svg)](/Teaching//assignments/assignment-performance-evaluation.ipynb)
Group Name:

Student Name | Student Number
--- | ---
**1:** | &#32;
**2:** | &#32;
**3:** | &#32;
**4:** | &#32;
**5:** | &#32;


This is an assignment. You should work in groups. Please write your group and group member names above. You will find sections labeled **Task** asking you to do each piece of analysis. Please make sure that you complete all of these tasks. I included some tests to help you see if you are calculating the solution correctly, but if you cannot get the test to pass submit your best attempt and you may recieve partial credit.

All work that you submit should be your own. Make use of the course resources and example code on the course website. It should be possible to complete all the requested tasks using information given below or somewhere on the course website.

For testing



In [2]:
#r "nuget: FsUnit.Xunit"
#r "nuget: xunit, 2.*"
open Xunit
open FsUnit.Xunit
open FsUnitTyped


Installed Packages FsUnit.Xunit, 5.3.0 xunit, 2.4.2

For the assignment



In [3]:
#r "nuget:FSharp.Data"
#r "nuget: FSharp.Stats"
#r "nuget: NovaSBE.Finance, 0.2.0-beta1"
#r "nuget: MathNet.Numerics"
#r "nuget: MathNet.Numerics.FSharp"
#r "nuget: Plotly.NET, 3.*"


Installed Packages FSharp.Data, 6.2.0 FSharp.Stats, 0.4.11 FsUnit.Xunit, 5.3.0 MathNet.Numerics, 5.0.0 MathNet.Numerics.FSharp, 5.0.0 NovaSBE.Finance, 0.2.0-beta1 Plotly.NET, 3.0.1 xunit, 2.4.2

In [4]:
#r "nuget: Plotly.NET.Interactive, 3.*"


Installed Packages FSharp.Data, 6.2.0 FSharp.Stats, 0.4.11 FsUnit.Xunit, 5.3.0 MathNet.Numerics, 5.0.0 MathNet.Numerics.FSharp, 5.0.0 NovaSBE.Finance, 0.2.0-beta1 Plotly.NET, 3.0.1 Plotly.NET.Interactive, 3.0.2 xunit, 2.4.2

Loading extensions from `/Users/martins/.nuget/packages/plotly.net.interactive/3.0.2/interactive-extensions/dotnet/Plotly.NET.Interactive.dll`

In [5]:
open System
open FSharp.Data
open Plotly.NET
open FSharp.Stats
open MathNet.Numerics.Statistics
open NovaSBE.Finance.Ols


## Load Data

First, make sure that you're referencing the correct files.

Here I'm assuming that you have a class folder with this notebook and a `data` folder inside of it. The folder hierarchy would look like below where you
have the below files and folders accessible:

```code
/class
    notebook.ipynb
    id_and_return_data.csv
    rd_sale.csv
    
```


In [6]:
open NovaSBE.Finance.Portfolio


### Data files

We assume the `id_and_return_data.csv` file and the signal csv file  are in the notebook folder.



In [7]:
let [<Literal>] IdAndReturnsFilePath = "id_and_return_data.csv"
let [<Literal>] MySignalFilePath = "eqnpo_me.csv"
let strategyName = "Net Payout Yield"


If my paths are correct, then this code should read the first few lines of the files.
If it doesn't show the first few lines, fix the above file paths.



In [9]:
IO.File.ReadLines(IdAndReturnsFilePath) 
|> Seq.truncate 5
|> Seq.iter (printfn "%A")


"id(string),eom(date),source(string),sizeGrp(string),obsMain(bool),exchMain(bool),primarySec(bool),gvkey(string),iid(string),permno(int Option),permco(int Option),excntry(string),curcd(string),fx(string),common(bool),compTpci(string),crspShrcd(int Option),compExchg(string),crsp_exchcd(int Option),adjfct(float Option),shares(float Option),gics(int Option),sic(int Option),naics(int Option),ff49(int Option),ret(float Option),retExc(float Option),prc(float Option),marketEquity(float Option)"
"crsp_86432,2000-01-31T00:00:00.0000000,CRSP,micro,1,1,true,115876,01,86432,16313,USA,USD,1,true,,11,,3,2,5.218,40101010,6020,522110,45,-0.003906,-0.00824925,15.9375,83.161875"
"crsp_85640,2000-01-31T00:00:00.0000000,CRSP,small,1,1,true,002193,01,85640,20300,USA,USD,1,true,,11,,1,1,102.496,35102020,8051,623110,11,-0.157143,-0.161485863,3.6875,377.954"
"crsp_86430,2000-01-31T00:00:00.0000000,CRSP,micro,1,1,true,115946,01,86430,16319,USA,USD,1,true,,11,,3,1,10.764,45103010,7372,511210,36,0.285714,0.28137

In [10]:
IO.File.ReadLines(MySignalFilePath) 
|> Seq.truncate 5
|> Seq.iter (printfn "%A")


"id(string),eom(date),signal(float option)"
"comp_001034_01,2008-12-31T00:00:00.0000000,0.0129194801"
"comp_001043_01,2000-01-31T00:00:00.0000000,0"
"comp_001076_02,2010-12-31T00:00:00.0000000,0.0017061186"
"comp_001081_01,2007-10-31T00:00:00.0000000,0.0189993385"


Ok, now assuming those paths were correct the below code will work.
I will put all this prep code in one block so that it is easy to run.



In [11]:
let idAndReturnsCsv = 
    CsvProvider<IdAndReturnsFilePath,ResolutionFolder = __SOURCE_DIRECTORY__>.GetSample().Rows 
    |> Seq.toList
let mySignalCsv = 
    CsvProvider<MySignalFilePath,ResolutionFolder = __SOURCE_DIRECTORY__>.GetSample().Rows 
    |> Seq.toList


A list of `Signal` records. The signal type is defined in the `NovaSBE.Finance.Portfolio` module [here](https://github.com/nhirschey/NovaSBE.Finance/blob/6d1398625e5a9279af00bb6e1c1802af3596c3f6/src/NovaSBE.Finance/Portfolio.fs#L178-L181).



In [12]:
let mySignals =
    mySignalCsv
    |> List.choose (fun row -> 
        match row.Signal with
        | None -> None
        | Some signal ->
            let signalRecord: Signal =
                { SecurityId = Other row.Id
                  FormationDate = DateTime(row.Eom.Year, row.Eom.Month, 1)
                  Signal = signal }
            Some signalRecord)

// look at a few signals
mySignals[..3]


index value 0 { SecurityId = Other "comp_001034_01"\n FormationDate = 01/12/2008 00:00:00\n Signal = 0.0129194801 } SecurityId Other "comp_001034_01" Item comp_001034_01 FormationDate 2008-12-01 00:00:00Z Signal 0.0129194801 1 { SecurityId = Other "comp_001043_01"\n FormationDate = 01/01/2000 00:00:00\n Signal = 0.0 } SecurityId Other "comp_001043_01" Item comp_001043_01 FormationDate 2000-01-01 00:00:00Z Signal 0 2 { SecurityId = Other "comp_001076_02"\n FormationDate = 01/12/2010 00:00:00\n Signal = 0.0017061186 } SecurityId Other "comp_001076_02" Item comp_001076_02 FormationDate 2010-12-01 00:00:00Z Signal 0.0017061186 3 { SecurityId = Other "comp_001081_01"\n FormationDate = 01/10/2007 00:00:00\n Signal = 0.0189993385 } SecurityId Other "comp_001081_01" Item comp_001081_01 FormationDate 2007-10-01 00:00:00Z Signal 0.0189993385

A list of Security return records. The `SecurityReturn` type is defined in the `NovaSBE.Finance.Portfolio` module [here](https://github.com/nhirschey/NovaSBE.Finance/blob/6d1398625e5a9279af00bb6e1c1802af3596c3f6/src/NovaSBE.Finance/Portfolio.fs#L173-L176)



In [13]:
let myReturns =
    idAndReturnsCsv
    |> List.choose (fun row -> 
        match row.Ret with
        | None -> None
        | Some ret ->
            let ret: SecurityReturn =
                { SecurityId = Other row.Id
                  Date = DateTime(row.Eom.Year, row.Eom.Month, 1)
                  Return= ret }
            Some ret)

// look at a few returns
myReturns[..3]


index value 0 { SecurityId = Other "crsp_86432"\n Date = 01/01/2000 00:00:00\n Return = -0.003906 } SecurityId Other "crsp_86432" Item crsp_86432 Date 2000-01-01 00:00:00Z Return -0.003906 1 { SecurityId = Other "crsp_85640"\n Date = 01/01/2000 00:00:00\n Return = -0.157143 } SecurityId Other "crsp_85640" Item crsp_85640 Date 2000-01-01 00:00:00Z Return -0.157143 2 { SecurityId = Other "crsp_86430"\n Date = 01/01/2000 00:00:00\n Return = 0.285714 } SecurityId Other "crsp_86430" Item crsp_86430 Date 2000-01-01 00:00:00Z Return 0.285714 3 { SecurityId = Other "crsp_85756"\n Date = 01/01/2000 00:00:00\n Return = 0.190685 } SecurityId Other "crsp_85756" Item crsp_85756 Date 2000-01-01 00:00:00Z Return 0.190685

A list of security market caps. We'll need this for value-weight portfolios. The `WeightVariable` type is defined in the `NovaSBE.Finance.Portfolio` module [here](https://github.com/nhirschey/NovaSBE.Finance/blob/6d1398625e5a9279af00bb6e1c1802af3596c3f6/src/NovaSBE.Finance/Portfolio.fs#L183-L186).



In [14]:
let myMktCaps =
    idAndReturnsCsv
    |> List.choose (fun row -> 
        match row.MarketEquity with
        | None -> None
        | Some mktCap ->
            let mktCap: WeightVariable =
                { SecurityId = Other row.Id
                  FormationDate = DateTime(row.Eom.Year, row.Eom.Month, 1)
                  Value = mktCap }
            Some mktCap)

// look at a few market caps
myMktCaps[..3]


index value 0 { SecurityId = Other "crsp_86432"\n FormationDate = 01/01/2000 00:00:00\n Value = 83.161875 } SecurityId Other "crsp_86432" Item crsp_86432 FormationDate 2000-01-01 00:00:00Z Value 83.161875 1 { SecurityId = Other "crsp_85640"\n FormationDate = 01/01/2000 00:00:00\n Value = 377.954 } SecurityId Other "crsp_85640" Item crsp_85640 FormationDate 2000-01-01 00:00:00Z Value 377.954 2 { SecurityId = Other "crsp_86430"\n FormationDate = 01/01/2000 00:00:00\n Value = 42.38325 } SecurityId Other "crsp_86430" Item crsp_86430 FormationDate 2000-01-01 00:00:00Z Value 42.38325 3 { SecurityId = Other "crsp_85756"\n FormationDate = 01/01/2000 00:00:00\n Value = 16.2155 } SecurityId Other "crsp_85756" Item crsp_85756 FormationDate 2000-01-01 00:00:00Z Value 16.2155

## Forming our strategy

We're now going to use the `Backtest` code to generate portfolios. It is defined in the `NovaSBE.Finance.Portfolio` module [here](https://github.com/nhirschey/NovaSBE.Finance/blob/6d1398625e5a9279af00bb6e1c1802af3596c3f6/src/NovaSBE.Finance/Portfolio.fs#L199).
The `Backtest` class automates some of the code we did earlier to make portfolio construction simpler.



In [13]:
let backtest = Backtest(returns=myReturns, signals=mySignals, nPortfolios=3, name = strategyName)


### Value Weighted Portfolios



In [14]:
let vw = backtest.strategyValueWeighted(myMktCaps)

vw.Portfolios[..3]


index value 0 { Name = "R&D to sales"\n Index = 1\n FormationMonth = 01/12/2008 00:00:00\n Positions =\n [{ SecurityId = Other "crsp_92788"\n Weight = 6.011819402e-06 }; { SecurityId = Other "crsp_89889"\n Weight = 4.113690326e-06 };\n { SecurityId = Other "crsp_844... Name R&D to sales Index 1 FormationMonth 2008-12-01 00:00:00Z Positions index value 0 { SecurityId = Other "crsp_92788"\n Weight = 6.011819402e-06 } SecurityId Other "crsp_92788" Item crsp_92788 Weight 6.011819402410821E-06 1 { SecurityId = Other "crsp_89889"\n Weight = 4.113690326e-06 } SecurityId Other "crsp_89889" Item crsp_89889 Weight 4.113690326282513E-06 2 { SecurityId = Other "crsp_84419"\n Weight = 1.955907925e-06 } SecurityId Other "crsp_84419" Item crsp_84419 Weight 1.9559079251269987E-06 3 { SecurityId = Other "comp_013520_01"\n Weight = 1.336117219e-05 } SecurityId Other "comp_013520_01" Item comp_013520_01 Weight 1.3361172190407486E-05 4 { SecurityId = Other "comp_061586_01"\n Weight = 1.876532313e-06 } SecurityId Other "comp_061586_01" Item comp_061586_01 Weight 1.8765323133621455E-06 5 { SecurityId = Other "comp_180759_01"\n Weight = 3.898808365e-05 } SecurityId Other "comp_180759_01" Item comp_180759_01 Weight 3.898808365152524E-05 6 { SecurityId = Other "crsp_10028"\n Weight = 6.870582688e-06 } SecurityId Other "crsp_10028" Item crsp_10028 Weight 6.870582688010961E-06 7 { SecurityId = Other "crsp_10514"\n Weight = 1.683447986e-05 } SecurityId Other "crsp_10514" Item crsp_10514 Weight 1.683447986019152E-05 8 { SecurityId = Other "crsp_10693"\n Weight = 0.0003964753773 } SecurityId Other "crsp_10693" Item crsp_10693 Weight 0.0003964753773205057 9 { SecurityId = Other "crsp_11161"\n Weight = 1.046455125e-06 } SecurityId Other "crsp_11161" Item crsp_11161 Weight 1.0464551251566632E-06 10 { SecurityId = Other "crsp_11267"\n Weight = 0.0001715506136 } SecurityId Other "crsp_11267" Item crsp_11267 Weight 0.00017155061361068873 11 { SecurityId = Other "crsp_11379"\n Weight = 8.852578653e-06 } SecurityId Other "crsp_11379" Item crsp_11379 Weight 8.852578652683166E-06 12 { SecurityId = Other "crsp_11382"\n Weight = 0.000255739051 } SecurityId Other "crsp_11382" Item crsp_11382 Weight 0.0002557390510333805 13 { SecurityId = Other "crsp_11402"\n Weight = 6.014983554e-06 } SecurityId Other "crsp_11402" Item crsp_11402 Weight 6.014983554330307E-06 14 { SecurityId = Other "crsp_11510"\n Weight = 8.19289123e-06 } SecurityId Other "crsp_11510" Item crsp_11510 Weight 8.192891229704962E-06 15 { SecurityId = Other "crsp_11618"\n Weight = 0.002127327242 } SecurityId Other "crsp_11618" Item crsp_11618 Weight 0.002127327242279367 16 { SecurityId = Other "crsp_11891"\n Weight = 0.001563625419 } SecurityId Other "crsp_11891" Item crsp_11891 Weight 0.0015636254194532145 17 { SecurityId = Other "crsp_12173"\n Weight = 0.0001662955403 } SecurityId Other "crsp_12173" Item crsp_12173 Weight 0.00016629554026052792 18 { SecurityId = Other "crsp_14461"\n Weight = 2.077542214e-05 } SecurityId Other "crsp_14461" Item crsp_14461 Weight 2.0775422139279543E-05 19 { SecurityId = Other "crsp_14868"\n Weight = 0.0001636852229 } SecurityId Other "crsp_14868" Item crsp_14868 Weight 0.00016368522291861155 ... (more) 1 { Name = "R&D to sales"\n Index = 2\n FormationMonth = 01/12/2008 00:00:00\n Positions =\n [{ SecurityId = Other "crsp_60206"\n Weight = 0.0009004373106 }; { SecurityId = Other "crsp_12052"\n Weight = 0.008937442865 };\n { SecurityId = Other "crsp_4292... Name R&D to sales Index 2 FormationMonth 2008-12-01 00:00:00Z Positions index value 0 { SecurityId = Other "crsp_60206"\n Weight = 0.0009004373106 } SecurityId Other "crsp_60206" Item crsp_60206 Weight 0.0009004373105997777 1 { SecurityId = Other "crsp_12052"\n Weight = 0.008937442865 } SecurityId Other "crsp_12052" Item crsp_12052 Weight 0.008937442864659815 2 { SecurityId = Other "crsp_42922"\n Weight = 3.067289756e-05 } SecurityId Other "crsp_42922" Item crsp_42922 Weight 3.0672897561809436E-05 3 { SecurityId = Other "c

In [15]:
vw.Returns[..3]


index value 0 { Name = "R&D to sales"\n Index = 1\n Month = 01/01/2009 00:00:00\n Return = -0.07654766749 } Name R&D to sales Index 1 Month 2009-01-01 00:00:00Z Return -0.07654766748807555 1 { Name = "R&D to sales"\n Index = 2\n Month = 01/01/2009 00:00:00\n Return = -0.04893451719 } Name R&D to sales Index 2 Month 2009-01-01 00:00:00Z Return -0.048934517187427515 2 { Name = "R&D to sales"\n Index = 3\n Month = 01/01/2009 00:00:00\n Return = -0.0461160921 } Name R&D to sales Index 3 Month 2009-01-01 00:00:00Z Return -0.04611609210452799 3 { Name = "R&D to sales"\n Index = 1\n Month = 01/02/2000 00:00:00\n Return = -0.04733696242 } Name R&D to sales Index 1 Month 2000-02-01 00:00:00Z Return -0.047336962422449766

### Long-short portfolios

We get the Fama-French 3-Factor asset pricing model data.



In [16]:
open NovaSBE.Finance

let ff3Lookup = 
    French.getFF3 French.Frequency.Monthly
    |> Array.map (fun x -> DateTime(x.Date.Year, x.Date.Month, 1), x)
    |> Map


Error: System.TypeLoadException: Could not load type 'FSharp.Data.Runtime.CsvFile`1' from assembly 'FSharp.Data, Version=6.2.0.0, Culture=neutral, PublicKeyToken=49286adf818aa259'.
   at NovaSBE.Finance.French.getFF3(Frequency frequency)
   at <StartupCode$FSI_0018>.$FSI_0018.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

Isolate some notable portfolios.



In [17]:
type SignalPortfolioObs = 
    { Month: DateTime
      Name: string
      ExcessReturn: float }
let long =
    vw.Returns
    |> List.filter (fun row -> row.Index = 3)
    |> List.map (fun row ->
        let retx = row.Return - ff3Lookup[row.Month].Rf 
        { Month = row.Month
          Name = "Long"
          ExcessReturn = retx })

let short =
    vw.Returns
    |> List.filter (fun row -> row.Index = 1)
    |> List.map (fun row -> 
        let retx = row.Return - ff3Lookup[row.Month].Rf 
        { Month = row.Month
          Name = "Short"
          ExcessReturn = retx })

let longShort =
    vw.Returns
    |> List.groupBy (fun x -> x.Month)
    |> List.map (fun (month, xs) ->
        let long = xs |> List.find (fun x -> x.Index = 3)
        let short = xs |> List.find (fun x -> x.Index = 1)
        { Month = long.Month
          Name = "Long-short"
          ExcessReturn = long.Return - short.Return })


Error: input.fsx (9,33)-(9,42) typecheck error The value or constructor 'ff3Lookup' is not defined.
input.fsx (18,33)-(18,42) typecheck error The value or constructor 'ff3Lookup' is not defined.

## Start of assignment

> **Task:** Calculate the annualized Sharpe ratios of your long, short, and long-short portfolios. Assign them to values `longSharpe`, `shortSharpe`, and `longShortSharpe`, respectively.
> 



In [18]:
// Solution here.


Tests



In [19]:
longSharpe |> should (equalWithin 1e-6) 0.2613490337
shortSharpe |> should (equalWithin 1e-6) 0.490118588
longShortSharpe |> should (equalWithin 1e-6) -0.05676492997


Error: input.fsx (1,1)-(1,11) typecheck error The value or constructor 'longSharpe' is not defined.
input.fsx (2,1)-(2,12) typecheck error The value or constructor 'shortSharpe' is not defined.
input.fsx (3,1)-(3,16) typecheck error The value or constructor 'longShortSharpe' is not defined.

> **Task:** Create a `list<RegData>` for your  long-short portfolio. Assign it to a value named `longShortRd`.
> 



In [20]:
type RegData =
    { Month: DateTime
      ExcessReturn: float 
      MktRf: float
      Hml: float
      Smb: float }

// Solution here.


Tests



In [21]:
longShortRd |> shouldHaveLength 252
longShortRd |> should be ofExactType<RegData list>


Error: input.fsx (1,1)-(1,12) typecheck error The value or constructor 'longShortRd' is not defined.
input.fsx (2,1)-(2,12) typecheck error The value or constructor 'longShortRd' is not defined.

> **Task:** Fit CAPM and Fama-French 3-factor models for your long-short portfolio. Assign them to a values named `capmModel` and `ff3Model`, respectively.
> 



In [22]:
// Solution here.


Tests.



In [23]:
capmModel |> should be ofExactType<RegressionResults>
ff3Model |> should be ofExactType<RegressionResults>


Error: input.fsx (1,1)-(1,10) typecheck error The value or constructor 'capmModel' is not defined.
input.fsx (2,1)-(2,9) typecheck error The value or constructor 'ff3Model' is not defined.

### CAPM model evaluation.

> **Task:** What is the CAPM alpha for your long-short portfolio? Use code to assign the alpha to the value `capmAlpha`. Is it significantly different from zero? Use code to assign the t-statistic to the value `capmAlphaT`.
> 



In [24]:
// Solution here.


Tests



In [25]:
capmAlpha |> should (equalWithin 1e-6) -0.003015844638
capmAlphT |> should (equalWithin 1e-6) -1.011538156


Error: input.fsx (1,1)-(1,10) typecheck error The value or constructor 'capmAlpha' is not defined.
input.fsx (2,1)-(2,10) typecheck error The value or constructor 'capmAlphT' is not defined.

> **Task:** What is the CAPM beta for your long-short portfolio? Use code to assign the beta to the value `capmBeta`. Is it significantly different from zero? Use code to assign the t-statistic to the value `capmBetaT`.
> 



In [26]:
// Solution here.


Tests



In [27]:
capmBeta |> should (equalWithin 1e-6) 0.3874415756
capmBetaT |> should (equalWithin 1e-6) 5.957967542


Error: input.fsx (1,1)-(1,9) typecheck error The value or constructor 'capmBeta' is not defined.
input.fsx (2,1)-(2,10) typecheck error The value or constructor 'capmBetaT' is not defined.

> **Task:** What is the information ratio for your long-short portfolio when using the CAPM model? Assign it to a value named `capmIR`.
> 



In [28]:
// Solution here.


Tests



In [29]:
capmIR |> should (equalWithin 1e-6) -0.06434136117


Error: input.fsx (1,1)-(1,7) typecheck error The value or constructor 'capmIR' is not defined.

### Fama-French 3-factor model evaluation.

> **Task:** What is the Fama-French 3-factor model alpha for your long-short portfolio. Is it significantly different from zero?
> 



In [30]:
// Solution here.


> **Task:** What are the betas on the Market, HML, and SMB factors for your long-short portfolio. Are they significantly different from zero?
> 



In [31]:
// Solution here.


> **Task:** Based on the Market, HML, and SMB factor betas for your long-short portfolio, would you say your portfolio is more like a value portfolio, more like a growth portfolio, or neither? Explain.
> 



In [32]:
// Solution here.


> **Task:** Based on the Market, HML, and SMB factor betas for your long-short portfolio, would you say your portfolio is more like a small-cap portfolio, more like a large-cap portfolio, or neither? Explain.
> 



In [33]:
// Solution here.


> **Task:** What is the information ratio for your long-short portfolio when using the Fama and French 3-factor model?
> 



In [34]:
// Solution here.
